In [2]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
from datetime import date

# Initialize Earth Engine
cloud_project = 'ee-projectfcf'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

# Create a map instance
Map = geemap.Map(center=[18.75, 73.7], zoom=10)

# Default date range
start_date = '2024-03-01'
end_date = '2024-04-01'

# Placeholder ROI (will be updated interactively)
roi = None

# Define ROI interaction widget
roi_button = widgets.Button(description="Define ROI", button_style='info')
roi_label = widgets.Label("Draw a region of interest (ROI) on the map, then click 'Define ROI'.")

def define_roi(b):
    global roi
    if Map.user_roi:
        roi = Map.user_roi
        Map.addLayer(roi, {}, "ROI")
        roi_label.value = "ROI defined! Continue by selecting a date range and loading images."
    else:
        roi_label.value = "No ROI detected. Please draw an ROI on the map and try again."

roi_button.on_click(define_roi)

# Date picker widgets for selecting date range
start_date_picker = widgets.DatePicker(
    description='Start Date',
    value=date(2024, 3, 1),  # Use the standard Python `datetime.date`
    disabled=False
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    value=date(2024, 4, 1),  # Use the standard Python `datetime.date`
    disabled=False
)

# Fetch and process Sentinel-2 Image Collection
def fetch_image_collection():
    global roi, start_date, end_date
    if not roi:
        raise ValueError("ROI is not defined. Please define an ROI first.")
    
    # Update start_date and end_date based on user input
    start_date = start_date_picker.value.strftime('%Y-%m-%d')
    end_date = end_date_picker.value.strftime('%Y-%m-%d')

    sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
        .filterDate(start_date, end_date) \
        .filterBounds(roi)

    # Get image IDs and dates
    images = sentinel2.toList(sentinel2.size())
    image_ids = [ee.Image(images.get(i)).get('system:index').getInfo() for i in range(images.size().getInfo())]
    image_dates = [
        ee.Date(ee.Image(images.get(i)).get('system:time_start')).format("YYYY-MM-dd").getInfo()
        for i in range(images.size().getInfo())
    ]

    return images, image_ids, image_dates

# Dropdown widget for image selection
dropdown = widgets.Dropdown(
    options=[],
    description='Select Image:',
    disabled=True
)

# Update map with NDVI visualization
def update_map(change):
    global images, roi
    Map.layers = Map.layers[:1]  # Clear all layers except ROI

    selected_index = dropdown.value
    selected_image = ee.Image(images.get(selected_index))

    # Calculate NDVI
    ndvi = selected_image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    # Visualization parameters
    vis_params_rgb = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.4
    }
    vis_params_ndvi = {
        'min': -1,
        'max': 1,
        'palette': ['blue', 'white', 'green']
    }

    # Add layers
    Map.addLayer(selected_image.clip(roi), vis_params_rgb, f'RGB - {image_dates[selected_index]}')
    Map.addLayer(ndvi.clip(roi), vis_params_ndvi, f'NDVI - {image_dates[selected_index]}')
    Map.add_colorbar(vis_params_ndvi, label="NDVI")

    Map.centerObject(roi, zoom=12)

dropdown.observe(update_map, names='value')

# Button to load images
load_button = widgets.Button(description="Load Images", button_style='primary')
load_label = widgets.Label("Click to load images after defining an ROI and date range.")

def load_images(b):
    global images, image_dates
    try:
        images, image_ids, image_dates = fetch_image_collection()
        dropdown.options = {f"{date} - {img_id}": i for i, (date, img_id) in enumerate(zip(image_dates, image_ids))}
        dropdown.disabled = False
        load_label.value = f"Loaded {len(image_ids)} images. Select one to view."
    except Exception as e:
        load_label.value = str(e)

load_button.on_click(load_images)

# Layout widgets and map
controls = widgets.VBox([
    roi_label,
    roi_button,
    widgets.Label("Select Date Range:"),
    start_date_picker,
    end_date_picker,
    load_label,
    load_button,
    dropdown
])

app_layout = widgets.HBox([controls, Map])

# Display the application
display(app_layout)
